# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/output_data/cityweatherdata.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,City,Country,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Latitude,Longitude
0,ushuaia,AR,33.80,69,40,19.46,-54.80,-68.30
1,bethel,US,72.00,88,1,2.77,41.37,-73.41
2,freeport,US,77.00,88,20,3.51,40.66,-73.58
3,cape town,ZA,63.00,63,75,8.05,-33.93,18.42
4,shangrao,CN,82.47,79,100,1.28,28.45,117.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity (%)"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
# Save the figure
plt.savefig("output_data/heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temperature"] < 80) & (filtered_weather_df["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed (MPH)"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Country,Max Temperature,Humidity (%),Cloudiness (%),Wind Speed (MPH),Latitude,Longitude
19,dutlwe,BW,72.14,19,0,8.14,-23.98,23.90
77,lac du bonnet,CA,73.00,59,0,8.99,50.25,-96.06
111,richards bay,ZA,74.28,49,0,9.15,-28.78,32.04
164,sinnamary,GF,79.95,83,0,5.06,5.38,-52.95
211,indiaroba,BR,78.19,79,0,6.17,-11.52,-37.51
212,karratha,AU,71.46,47,0,5.61,-20.74,116.85
439,novo aripuana,BR,78.37,67,0,2.19,-5.12,-60.38
470,ghanzi,BW,74.70,17,0,5.28,-21.57,21.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 19: dutlwe.
Missing field/result... skipping.
------------
Retrieving Results for Index 77: lac du bonnet.
Closest hotel is Casey's Inn Motor Hotel.
------------
Retrieving Results for Index 111: richards bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 164: sinnamary.
Closest hotel is ALETHEIA Appartement Sinnamary.
------------
Retrieving Results for Index 211: indiaroba.
Closest hotel is Igreja de Jesus Cristo.
------------
Retrieving Results for Index 212: karratha.
Closest hotel is ibis Styles Karratha.
------------
Retrieving Results for Index 439: novo aripuana.
Closest hotel is Hotel Ypê.
------------
Retrieving Results for Index 470: ghanzi.
Closest hotel is Ghanzi Farmhouse.
------------


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Add marker layer ontop of heat map
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Save the figure
plt.savefig("output_data/hotelmap.png")
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>